In [ ]:
!pip install geopandas
!pip install h3

     |████████████████████████████████| 972kB 2.9MB/s 
     |████████████████████████████████| 14.7MB 243kB/s 
     |████████████████████████████████| 10.9MB 45.4MB/s 
     |████████████████████████████████| 768kB 2.8MB/s 


In [ ]:
import geopandas as gpd
import pandas as pd
import os
from h3 import h3
import json
from tqdm import tqdm
import pyarrow.parquet as pq
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
def explode(indata):
    indf = indata
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

def reverse_lat_lon(hex_coords):
    geom_hex = []
    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1],lat_lon[0]])

    return geom_hex



def get_hex_cover(geojson_layer):
    hex_list = []

    for i in tqdm(range(0,len(geojson_layer["features"]),1)):
        set_hexagons = h3.polyfill(geojson = geojson_layer["features"][i]["geometry"], res = 4, geo_json_conformant = True)
        list_hexagons = list(set_hexagons)
        hex_list.append(list_hexagons)

    flat_list = [item for sublist in hex_list for item in sublist]

    df_fill_hex = pd.DataFrame({"hex_id": flat_list})
    df_fill_hex["value"] = 0
    df_fill_hex['geometry'] = df_fill_hex["hex_id"].apply(lambda x: Polygon([reverse_lat_lon(h3.h3_to_geo_boundary(h=x,geo_json=False))][0]))
    df_fill_hex = gpd.GeoDataFrame(df_fill_hex)

    df_fill_hex["hex_center"] = df_fill_hex.apply(lambda row: row["geometry"].centroid, axis=1)
    df_fill_hex["hex_lon"] = df_fill_hex.apply(lambda row: row["hex_center"].x, axis=1)
    df_fill_hex["hex_lat"] = df_fill_hex.apply(lambda row: row["hex_center"].y, axis=1)
    return df_fill_hex



In [ ]:
table2 = pq.read_table('/Users/restore/Downloads/population_v3.parquet')
df = table2.to_pandas()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lat_round'], df['lon_round'])) # fix this

In [ ]:
# Load Shapefiles
us_gdf = gpd.read_file('https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_20m.json')
us_gdf = us_gdf.loc[~(us_gdf['NAME'].isin(['Hawaii','Alaska']))]
us_gdf = explode(us_gdf)


In [ ]:
geojson_layer = json.loads(us_gdf.to_json())

In [ ]:
df_fill_hex = get_hex_cover(geojson_layer)

100%|██████████| 166/166 [00:00<00:00, 617.34it/s]


In [ ]:
df_fill_hex.head()

,hex_id,value,geometry,hex_center,hex_lon,hex_lat
0,8448cb5ffffffff,0,"POLYGON ((-109.29491 36.04102, -109.04849 36.1...",POINT (-109.3385842615073 36.28555017540165),-109.338584,36.285550
1,84269a7ffffffff,0,"POLYGON ((-109.96540 36.71093, -109.71850 36.8...",POINT (-110.0108183913315 36.95392193295385),-110.010818,36.953922
2,8429b1dffffffff,0,"POLYGON ((-114.43740 34.58058, -114.20352 34.7...",POINT (-114.4912927964836 34.82446133053896),-114.491293,34.824461
3,8448cb1ffffffff,0,"POLYGON ((-109.83025 35.97852, -109.58479 36.1...",POINT (-109.8750665338222 36.22292858735794),-109.875067,36.222929
4,8448ea5ffffffff,0,"POLYGON ((-109.50349 32.60471, -109.26481 32.7...",POINT (-109.5462861277193 32.85444891790489),-109.546286,32.854449


In [ ]:
gdf = gpd.sjoin(df_fill_hex, df, how='left',op='intersects',)